In [ ]:
from mycelia.shared.model import load_base_model
from mycelia.shared.modeling.modeling_mycelia import get_base_tokenizer
from mycelia.config import Config
from mycelia.shared.datasets import get_dataloader
from mycelia.shared.logging import configure_logging
from mycelia.shared.helper import get_nested_attr

configure_logging()

config = Config()
config.model.device = "cuda:2"


In [13]:

config.moe.partial_moe

True

In [7]:
model, em = load_base_model(config = config, rank = 0)
# tokenizer = get_base_tokenizer(config)
# dataloader = get_dataloader(
#     config,
#     rank= config.data.rank,
#     world_size= config.data.world_size,
#     tokenizer=tokenizer,
# )
# data_iter = iter(dataloader)

2025-10-07 02:21:20 [info     ] rank 0: Checkpoint path checkpoints/centralised not found, starting from scratch [mycelia.shared.checkpoint]


Loading checkpoint shards: 100%|██████████| 7/7 [00:07<00:00,  1.12s/it]


Instantiating DeepseekAttention without passing `layer_idx` is not recommended and will to errors during the forward call, if caching is used. Please make sure to provide a `layer_idx` when creating this class.
2025-10-07 02:22:22 [info     ] em init                      [mycelia.shared.expert_manager]
2025-10-07 02:22:22 [info     ] em get expert layer          [mycelia.shared.expert_manager]
2025-10-07 02:22:22 [info     ] em get expert layer 1        [mycelia.shared.expert_manager]
2025-10-07 02:22:22 [info     ] Detected expert layerss: [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27] [mycelia.shared.expert_manager]
2025-10-07 02:22:22 [info     ] seed 0 Expert group assignment for layer 27: {0: [1, 4, 5, 3], 1: [7, 2, 6, 0]} [mycelia.shared.expert_manager]


Loading checkpoint shards: 100%|██████████| 7/7 [00:07<00:00,  1.02s/it]


In [11]:
import torch
import torch.nn as nn

def count_parameters(model: nn.Module, trainable_only: bool = False):
    """
    Count parameters in a PyTorch model.

    Args:
        model (nn.Module): The model.
        trainable_only (bool): If True, count only parameters that require gradients.

    Returns:
        int: Total number of parameters.
    """
    if trainable_only:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in model.parameters())

count_parameters(model) / 1e9

1.79590144

In [ ]:
from transformers import (
    AutoConfig,
    AutoTokenizer,
    LlamaConfig,
    LlamaForCausalLM,
    OlmoForCausalLM,
    AutoModelForCausalLM,
    PretrainedConfig
)
org_model = AutoModelForCausalLM.from_pretrained(config.model.model_path, trust_remote_code=True)
output = org_model(**batch_device)

Loading checkpoint shards: 100%|██████████| 7/7 [01:48<00:00, 15.56s/it]


In [21]:

batch = next(data_iter)  

batch_device = {}
for key in batch.keys():
    batch_device[key] = batch[key].to(model.device)